In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [7]:
class MAML:
    def __init__(self,model,in_lr,out_lr,inner_steps,task_loss_fn):
        self.model=model
        self.in_lr=in_lr
        self.out_lr=out_lr
        self.inner_steps=inner_steps
        self.task_loss_fn=task_loss_fn
        self.outer_optim = optim.Adam(self.model.parameters(), lr=self.out_lr)
    
    def clone_curr_par(self):
        parameters_model={name:param.clone() for name,param in self.model.named_parameters()}
        
        return parameters_model
    
    def forward(self,X,task_param):
        for name,param in self.model.named_parameters():
            param.data=task_param[name]
        return self.model(X)
    
    def inner_loop(self,task_data,task_label):
        curr_parameters=self.clone_curr_par()
        
        for _ in range(self.inner_steps):
            pred=self.forward(task_data,curr_parameters)
            loss=self.task_loss_fn(pred,task_label)
            
            grad=torch.autograd.grad(loss,curr_parameters.value(),create_graph=True)
            curr_parameters={name:param-(self.inner_loop*grad) for (name,param),grad in zip(curr_parameters.items(),grad)}
        return curr_parameters
    
    def meta_train(self,task_set,validation_set):
        for task_data,task_label in task_set:
            task_param=self.inner_loop(task_data,task_label)
            validation_data,validation_label=validation_set
            pred=self.forward(validation_data,task_param)
            loss=self.task_loss_fn(pred,validation_label)
            
            self.outer_optim.zero_grad()
            loss.back_ward()
            self.outer_optim.step()
            return
        
    def meta_test(self,test_set):
        test_data,test_label=test_set
        new_param=self.inner_loop(test_data,test_label)
        pred=self.forward(test_data,new_param)
        
        return pred